In [1]:
from impala.dbapi import connect

conn = connect(host='alpha.master.hadoop.local', 
               port=10000, 
               database='ohs_stockdata_jp_test' or 'default', 
               auth_mechanism="PLAIN", 
               timeout=5,
               user='hadoop',
               password='hadoop')

cur = conn.cursor()
 
cur.execute('SHOW DATABASES')
print(cur.fetchall())
 
#cur.execute('SHOW Tables')
#print(cur.fetchall())


[('default',), ('ohs_stockdata',), ('ohs_stockdata_hk',), ('ohs_stockdata_hk_test',), ('ohs_stockdata_jp',), ('ohs_stockdata_jp_test',), ('ohs_stockdata_test',), ('ohs_stockdata_us',), ('ohs_stockdata_us_test',), ('plreport',)]


In [3]:
import pandas as pd
from impala.util import as_pandas

cur.execute('SELECT * FROM fi_quote_daily_tse_8609')
df = as_pandas(cur)

df

HiveServer2Error: Invalid SessionHandle: SessionHandle [b9885bc1-dc0f-4423-b649-0b358962cd6a]

In [5]:
conn.close()

In [1]:
import talib
import numpy as np
import pandas as pd
from pyecharts import Grid, Bar, Line, Kline, Overlap
from impala.dbapi import connect
from impala.util import as_pandas

conn = connect(host='alpha.master.hadoop.local', 
               port=10000, 
               database='ohs_stockdata_jp_test' or 'default', 
               auth_mechanism="PLAIN", 
               timeout=5,
               user='hadoop',
               password='hadoop')

cur = conn.cursor()
cur.execute('SELECT * FROM fi_quote_daily_tse_8609')

data = as_pandas(cur)
data.columns=['date','open','high','low','close','volume','openint']
cur.close()

#转换类型
ochl = data[['open', 'close', 'high', 'low']] .astype('float')
ochl_tolist = [ochl.iloc[i].tolist() for i in range(len(ochl))]

np_real_datas = np.array(data['close'], dtype='f8')

#转换real为double
sma_10 = talib.SMA(np_real_datas, 10)
sma_30 = talib.SMA(np_real_datas, 30)

kline = Kline()
kline.add("daily", data['date'], ochl_tolist, is_datazoom_show=True)

line = Line()
line.add('10d MA', data['date'], sma_10, is_fill=False, line_opacity=0.8, is_smooth=True)
line.add('30d MA', data['date'], sma_30, is_fill=False, line_opacity=0.8, is_smooth=True)

#bar = Bar()
#bar.add("Volume", data['date'], data['volume'], is_stack=True)

overlap = Overlap()
overlap.add(kline)
overlap.add(line)
#overlap.add(bar)

overlap